# Hyperparameters

In [88]:
MAX_ROWS_PARAMETER = 100
TRAINING_PERCENT_PARAMETER = 0.3
# RIDGE_ALPHA_PARAMETER = 10000.0
DEBUG_PARAMETER = False

# Preparing Data

In [146]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
from sklearn.utils import shuffle

from sklearn import tree

plays_path = '/Users/davidstange/Development/github/davestange/data/big_data_bowl_2025/plays.csv'

### Investigation

In [205]:
plays = pd.read_csv(plays_path)

plays['offenseFormation'].unique() # 'EMPTY', 'SHOTGUN', 'PISTOL', 'SINGLEBACK', nan, 'JUMBO', 'I_FORM', 'WILDCAT'
plays['receiverAlignment'].unique() # '3x2', '2x2', '3x1', nan, '4x1', '2x1', '1x1', '2x0', '3x0', '1x0', '4x2', '3x3'
plays['passResult'].unique() # 

# -------- IDENTIFIERS
# plays['gameId'].unique() # gameId
# plays['playId'].unique() # playId


# -------- PRE-SNAP DETAILS
# plays['quarter'].unique() # quarter
# plays['down'].unique() # down
# plays['yardsToGo'].unique() # yardsToGo
# plays['possessionTeam'].unique() # possessionTeam
# plays['defensiveTeam'].unique() # defensiveTeam
# plays['yardlineSide'].unique()            # what does this mean? which side of the field they are on?
# plays['yardlineNumber'].unique()          # 1-50
# plays['gameClock'].unique() # gameClock
# plays['preSnapHomeScore'].unique() # preSnapHomeScore
# plays['preSnapVisitorScore'].unique() # preSnapVisitorScore
# plays['absoluteYardlineNumber'].unique()  # 1-99
# plays['preSnapHomeTeamWinProbability'].unique() # preSnapHomeTeamWinProbability
# plays['preSnapVisitorTeamWinProbability'].unique() # preSnapVisitorTeamWinProbability
# plays['offenseFormation'].unique()        # 'EMPTY', 'SHOTGUN', 'PISTOL', 'SINGLEBACK', nan, 'JUMBO', 'I_FORM', 'WILDCAT'
# plays['receiverAlignment'].unique()       # '3x2', '2x2', '3x1', nan, '4x1', '2x1', '1x1', '2x0', '3x0', '1x0', '4x2', '3x3'
# plays['playClockAtSnap'].unique()         # playClockAtSnap


# -------- PLAY TYPE
# plays['qbSpike'].unique() # qbSpike
# plays['qbKneel'].unique() # qbKneel
# plays['qbSneak'].unique() # qbSneak
# plays['isDropback'].unique() # isDropback


# -------- POST-SNAP RESULT
# plays['playDescription'].unique() # playDescription



# plays['playNullifiedByPenalty'].unique() # playNullifiedByPenalty
# plays['expectedPoints'].unique() # expectedPoints
# plays['passResult'].unique() # passResult
# plays['passLength'].unique() # passLength
# plays['targetX'].unique() # targetX
# plays['targetY'].unique() # targetY
# plays['playAction'].unique() # playAction
# plays['dropbackType'].unique() # dropbackType
# plays['dropbackDistance'].unique() # dropbackDistance
# plays['passLocationType'].unique() # passLocationType
# plays['timeToThrow'].unique() # timeToThrow
# plays['timeInTackleBox'].unique() # timeInTackleBox
# plays['timeToSack'].unique() # timeToSack
# plays['passTippedAtLine'].unique() # passTippedAtLine
# plays['unblockedPressure'].unique() # unblockedPressure
# plays['rushLocationType'].unique()        # nan, 'INSIDE_LEFT', 'INSIDE_RIGHT', 'OUTSIDE_LEFT', 'UNKNOWN', 'OUTSIDE_RIGHT'
# plays['penaltyYards'].unique() # penaltyYards
# plays['prePenaltyYardsGained'].unique() # prePenaltyYardsGained
# plays['yardsGained'].unique() # yardsGained
# plays['homeTeamWinProbabilityAdded'].unique() # homeTeamWinProbabilityAdded
# plays['visitorTeamWinProbilityAdded'].unique() # visitorTeamWinProbilityAdded
# plays['expectedPointsAdded'].unique() # expectedPointsAdded
# plays['pff_runConceptPrimary'].unique()   # nan, 'MAN', 'INSIDE ZONE', 'UNDEFINED', 'PULL LEAD', 'OUTSIDE ZONE', 'TRAP', 'SNEAK', 'POWER', 'COUNTER', 'DRAW', 'TRICK', 'FB RUN'
# plays['pff_runConceptSecondary'].unique() # [nan, 'READ OPTION', 'LEAD', 'PITCH', 'INVERTED;READ OPTION', 'SPLIT', 'MISDIRECTION', 'READ OPTION;SPLIT', 'LEAD;PITCH', 'CROSS LEAD;LEAD', 'CROSS LEAD', 'LEAD;SPLIT', 'PITCH;SPLIT',
#                                           # 'READ OPTION;SPEED OPTION', 'LEAD;READ OPTION;SPLIT', 'LEAD;QB RUNS', 'QB RUNS', 'LEAD;READ OPTION', 'CROSS LEAD;READ OPTION', 'CROSS LEAD;QB RUNS', 'MISDIRECTION;PITCH', 'LEAD;SPEED OPTION',
#                                           # 'READ OPTION;SPEED OPTION;SPLIT', 'INVERTED;READ OPTION;SPLIT','SPEED OPTION', 'BACKSIDE FOLD','INVERTED;READ OPTION;SPEED OPTION',
#                                           # 'CROSS LEAD;READ OPTION;SPLIT', 'CROSS LEAD;LEAD;READ OPTION','CROSS LEAD;INVERTED;READ OPTION', 'CROSS LEAD;SPLIT','CROSS LEAD;PITCH', 'CROSS LEAD;MISDIRECTION',
#                                           # 'MISDIRECTION;SPLIT', 'CROSS LEAD;READ OPTION;SPEED OPTION','LEAD;MISDIRECTION', 'QB RUNS;SPLIT', 'MISDIRECTION;QB RUNS','LEAD;READ OPTION;SPEED OPTION', 'LEAD;PITCH;SPLIT',
#                                           # 'CROSS LEAD;LEAD;QB RUNS', 'BACKSIDE FOLD;QB RUNS','BACKSIDE FOLD;PITCH', 'LEAD;LEAD']
# plays['pff_runPassOption'].unique()       # 0, 1
# plays['pff_passCoverage'].unique()        # 'Cover-3', 'Quarters', 'Cover-1', 'Cover 6-Left', 'Cover-2', nan, 'Cover-0', 'Cover-3 Seam', 'Cover-6 Right', 'Goal Line', 'Red Zone', '2-Man', 'Cover-3 Double Cloud', 'Bracket',
#                                           # 'Miscellaneous', 'Cover-3 Cloud Left', 'Prevent', 'Cover-1 Double', 'Cover-3 Cloud Right'
# plays['pff_manZone'].unique()             # 'Zone', 'Man', nan, 'Other'
plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,...,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone
1,2022091809,3698,(2:13) (Shotgun) J.Burrow pass short right to ...,4,1,10,CIN,DAL,CIN,8,...,4,0.002847,-0.002847,-0.240509,True,NaN,NaN,0,Quarters,Zone
2,2022103004,3146,(2:00) (Shotgun) D.Mills pass short right to D...,4,3,12,HOU,TEN,HOU,20,...,6,0.000205,-0.000205,-0.218480,True,NaN,NaN,0,Quarters,Zone
3,2022110610,348,(9:28) (Shotgun) P.Mahomes pass short left to ...,1,2,10,KC,TEN,TEN,23,...,4,-0.001308,0.001308,-0.427749,True,NaN,NaN,0,Quarters,Zone
4,2022102700,2799,(2:16) (Shotgun) L.Jackson up the middle to TB...,3,2,8,BAL,TB,TB,27,...,-1,0.027141,-0.027141,-0.638912,False,MAN,READ OPTION,0,Cover-1,Man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16119,2022110604,1051,(12:49) (Shotgun) T.Lawrence pass incomplete s...,2,3,4,JAX,LV,JAX,31,...,0,-0.024741,0.024741,-1.391687,True,NaN,NaN,0,Cover-2,Zone
16120,2022103005,3492,(12:32) (Shotgun) K.Cousins pass incomplete de...,4,1,10,MIN,ARI,MIN,25,...,0,-0.026580,0.026580,-0.503208,True,NaN,NaN,0,Cover-3,Zone
16121,2022092502,2337,(9:59) (Shotgun) P.Mahomes scrambles right end...,3,1,10,KC,IND,IND,13,...,10,-0.013790,0.013790,1.073898,True,NaN,NaN,0,Quarters,Zone
16122,2022091809,719,(:45) C.Rush pass incomplete deep right to C.L...,1,1,10,DAL,CIN,CIN,47,...,0,-0.011561,0.011561,-0.522397,True,UNDEFINED,NaN,0,Cover-3,Zone


## Readying the dataset

In [208]:
def get_position_features(s: pd.Series) -> pd.Series:
    ### determine wr, te, rb based on alignment; WRxTE
    wr_count = te_count = rb_count = 0
    extra_ol = s.offenseFormation == "JUMBO"
    if not isinstance(s.receiverAlignment, float):
        alignment = s.receiverAlignment.split('x')
        if len(alignment):
            wr_count = int(alignment[0])
            te_count = int(alignment[1])
            rb_count = 5 - wr_count - te_count
            if extra_ol and rb_count > 1:
                rb_count -= 1 

    return pd.Series([wr_count, te_count, rb_count, extra_ol], ['wr_count', 'te_count', 'rb_count', 'extra_ol'])
def get_time_features(s: pd.Series) -> pd.Series:
    ### determine total time remaining, is overtime, or within two mins 
    two_min_1st_half = 32 * 60
    two_min_2st_half = 2 * 60
    two_min_overtime = 68 * 60
    
    quarter = int(s.quarter)
    secs_remaining = math.nan
    is_inside_two_mins = is_overtime = False
    quarter_clock = s.gameClock.split(':')
    if len(quarter_clock) == 2:
        secs_in_quarter = int(quarter_clock[0]) * 60 + int(quarter_clock[1])
        if quarter <= 4:
            secs_remaining = (4 - quarter) * 15 * 60 + secs_in_quarter
        else:
            secs_remaining = secs_in_quarter
            is_overtime = True
        is_inside_two_mins = secs_in_quarter <= 120 and (quarter == 2 or quarter >= 4)

    return pd.Series([secs_remaining, is_inside_two_mins, is_overtime], ['secs_remaining', 'is_inside_two_mins', 'is_overtime'])

# v1 result - just sneak, kneel, spikie, pass, or run
plays['play_result'] = plays.apply(lambda p: 
                                   'qb_sneak' if p.qbSneak == True else 
                                   'qb_kneel' if p.qbKneel == True else 
                                   'qb_spike' if p.qbSpike == True else 
                                   'pass' if p.isDropback == True else 
                                   'run', axis = 1)

# get position counts (TE, WR, etc)
tmp = plays.apply(get_position_features, axis=1)
plays['wr_count'] = tmp['wr_count']
plays['te_count'] = tmp['te_count']
plays['rb_count'] = tmp['rb_count']
plays['extra_ol'] = tmp['extra_ol']

tmp = plays.apply(get_time_features, axis=1)
plays['secs_remaining'] = tmp['secs_remaining']
plays['is_inside_two_mins'] = tmp['is_inside_two_mins']
plays['is_overtime'] = tmp['is_overtime']

# # preSnapHomeScore, preSnapVisitorScore, need Home/Away 
# plays['off_score_lead'] = plays.apply(...)

# # preSnapHomeTeamWinProbability	preSnapVisitorTeamWinProbability, need Home/Away 
# plays['off_win_probability'] = plays.apply(...)

plays['off_yards_to_goal'] = plays.apply(lambda p: 100 - int(p.yardlineNumber) if p.yardlineSide == p.possessionTeam else int(p.yardlineNumber), axis = 1)

## Split data set

In [212]:
# pre_snap_features = ['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'absoluteYardlineNumber', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'offenseFormation', 'receiverAlignment', 'playClockAtSnap']
# play_result_features = ['playDescription', 'play_result', 'qbSpike', 'qbKneel', 'qbSneak', 'isDropback']
TRAINING_FEATURES = ['down', 'yardsToGo',                                                       # down and distance
                     'quarter', 'secs_remaining', 'is_inside_two_mins', 'is_overtime',          # game time
                     'wr_count', 'te_count', 'rb_count', 'extra_ol', 'offenseFormation']        # formations
TARGET_FEATURE = 'play_result'

all_indices = range(0, len(plays))
all_indices = shuffle(all_indices, random_state=27)

# # HACK: the full 30k recordset takes too long, scaling back to first 1000 records 
# if MAX_ROWS_PARAMETER > 0 and MAX_ROWS_PARAMETER < len(all_indices):
#     all_indices = all_indices[:MAX_ROWS_PARAMETER]

split_index = int(len(all_indices) * TRAINING_PERCENT_PARAMETER)

training_indices, dev_indices = np.split(all_indices, [split_index])
training = plays.iloc[training_indices]
develop = plays.iloc[dev_indices]

auto_X_train = training[TRAINING_FEATURES]
auto_X_test = develop[TRAINING_FEATURES]
auto_y_train = training[[TARGET_FEATURE]]
auto_y_test = develop[[TARGET_FEATURE]]
if DEBUG_PARAMETER:
    print(f'training dataset: {len(training)}, development dataset: {len(develop)}')

## Validating & Debugging Features

In [ ]:
tv_watching = ['gameId', 'playId', 'playDescription', 'quarter', 'gameClock', 'down', 'yardsToGo', 'off_yards_to_goal', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'preSnapHomeScore', 'preSnapVisitorScore', 'secs_remaining', 'is_inside_two_mins', 'is_overtime']

steelers_bengals = plays[plays.gameId == 2022091103].sort_values(by='playId')

steelers_bengals[tv_watching].head(20)
# steelers_bengals.head(20)




# plays[plays.pff_manZone != 'Zone'][plays.pff_manZone != 'Man'][plays.pff_manZone != 'Other'][['pff_manZone', 'playDescription']]

,gameId,playId,playDescription,quarter,gameClock,down,yardsToGo,off_yards_to_goal,absoluteYardlineNumber,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,secs_remaining,is_inside_two_mins,is_overtime
10263,2022091103,58,(15:00) (Shotgun) M.Trubisky pass short middle...,1,15:00,1,10,75,35,PIT,CIN,PIT,25,0,0,3600,False,False
1278,2022091103,82,"(14:39) (No Huddle, Shotgun) M.Trubisky pass s...",1,14:39,2,7,72,38,PIT,CIN,PIT,28,0,0,3579,False,False
2595,2022091103,111,(14:21) (Shotgun) M.Trubisky pass short left t...,1,14:21,3,2,67,43,PIT,CIN,PIT,33,0,0,3561,False,False
6254,2022091103,176,(13:31) (Shotgun) J.Burrow sacked at CIN 17 fo...,1,13:31,1,10,78,88,CIN,PIT,CIN,22,0,0,3511,False,False
9518,2022091103,195,(12:52) (Shotgun) J.Burrow pass short right in...,1,12:52,2,15,83,93,CIN,PIT,CIN,17,0,0,3472,False,False
6579,2022091103,253,(12:43) (Shotgun) J.Mixon left guard to CIN 30...,1,12:43,1,10,75,85,CIN,PIT,CIN,25,0,7,3463,False,False
5868,2022091103,274,(12:05) (No Huddle) J.Mixon left tackle to CIN...,1,12:05,2,5,70,80,CIN,PIT,CIN,30,0,7,3425,False,False
459,2022091103,295,"(11:34) (No Huddle, Shotgun) J.Burrow pass sho...",1,11:34,1,10,64,74,CIN,PIT,CIN,36,0,7,3394,False,False
7468,2022091103,319,(10:55) (No Huddle) J.Mixon left end to CIN 39...,1,10:55,2,7,61,71,CIN,PIT,CIN,39,0,7,3355,False,False
1398,2022091103,340,(10:13) (Shotgun) J.Burrow pass short left to ...,1,10:13,3,7,61,71,CIN,PIT,CIN,39,0,7,3313,False,False


## Create Model

In [213]:
# decision_training_features = 

# decision_tree_model = tree.DecisionTreeClassifier(criterion='entropy')
# decision_tree_model = decision_tree_model.fit(auto_X_train, auto_y_train)
auto_X_train

,down,yardsToGo,quarter,secs_remaining,is_inside_two_mins,is_overtime,wr_count,te_count,rb_count,extra_ol,offenseFormation
3370,1,10,1,3596,False,False,3,1,1,False,SINGLEBACK
15160,2,16,2,2375,False,False,2,2,1,False,SHOTGUN
9326,1,10,4,120,True,False,2,2,1,False,SHOTGUN
10755,3,3,4,253,False,False,3,1,1,False,SHOTGUN
14244,2,7,4,726,False,False,2,2,1,False,SHOTGUN
...,...,...,...,...,...,...,...,...,...,...,...
14314,1,10,4,432,False,False,3,1,1,False,SINGLEBACK
1769,2,8,4,316,False,False,2,2,1,False,SINGLEBACK
9925,2,1,1,3327,False,False,2,2,1,False,SHOTGUN
7913,1,10,4,868,False,False,2,1,2,False,SINGLEBACK
